In [ ]:
import random , string,time,math
import pandas as pd
# from sklearn.datasets import make_regression
from snowflake.snowpark import Session
from random import randint
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler

import os
from snowflake.ml.modeling.preprocessing import MinMaxScaler as MMS, OrdinalEncoder as OE
from snowflake.ml.modeling.pipeline import Pipeline
import io
import joblib
from snowflake.ml.modeling.metrics.correlation import correlation

In [ ]:
session=Session.builder.getOrCreate()

In [ ]:
X1,X2=[randint(2,100) for _ in range(10000)],[randint(2,100) for _ in range (10000)]
    # X,_=make_regression(n_samples=1000, n_features=2,noise=0.1,random_state=0)
X=pd.DataFrame(data={'N1':X1,'N2':X2})
# return X

cat_features={}
for c in ['C1','C2']:
    cat_features[c]=["".join(random.choices(string.ascii_uppercase,k=2)) for _ in range(X.shape[0])]
X=X.assign(**cat_features)

df=session.create_dataframe(X)
df.write.mode("overwrite").save_as_table("test.public.regression_dataset")

df.show()

In [ ]:
X=session.table('test.public.regression_dataset').to_pandas()

start_time= time.time()
X[['C1O','C2O']]=OrdinalEncoder().fit_transform(X[['C1','C2']])

X[['N1FO','N2FO','C1FO','C2FO']]=MinMaxScaler().fit_transform(X[['N1','N2','C1O','C2O']])

total_time=math.trunc(time.time()-start_time)

print(f"{total_time} seconds")

X.head()

## Using Snowflake ML

In [ ]:
df=session.table("test.public.regression_dataset")

In [ ]:
df.show()

In [ ]:
pipe=Pipeline(steps=[
    (
        "encoder", OE(
            input_cols=['C1','C2'],
            output_cols=['C1O','C2O']
        )
    ),
    (
        "scaler",MMS(
            input_cols=['N1','N2','C1O','C2O'],
            output_cols=['N1FO','N2FO','C1FO','C2FO']
        )
    )
])

In [ ]:
pipe.fit(df)

In [ ]:
df=pipe.transform(df)

In [ ]:
df.show()

## Saving the Model

In [ ]:
model_name="random_model.joblib"
stage_location="@int_stage/random_model"

model_path=os.path.join(stage_location,model_name)
model_path

In [ ]:
input_stream=io.BytesIO()
input_stream.name=model_name
joblib.dump(value=pipe,filename=input_stream)
session.file.put_stream(
    input_stream=input_stream, 
    stage_location=model_path,
    overwrite=True
)

In [ ]:
df_corr=correlation(df=df)

In [ ]:
df_corr